**О ДАТАСЕТЕ: Экспорт и импорт российских регионов: таможенная статистика с детализацией до товаров за 2016-2021 гг.**

Сведения Федеральной таможенной службы по экспорту и импорту из российских регионов за 2016-2021 годы. Данные представлены с детализацией до десятизначных кодов товаров, а также стран, в которые эти товары экспортируются или импортируются субъектами РФ. 

По каждой внешнеторговой поставке в Федеральной таможенной службе России заполняется таможенная декларация, содержащая информацию о товарах, которые иностранный контрагент купил у российской организации (или наоборот). Для публикации в открытом доступе данные таможенных деклараций укрупняются с уровня конкретных предприятий до уровня регионов России. Датасет содержит информацию о стране поставки, коде товара, стоимости поставки в долларах, весе нетто поставки в килограммах, объеме поставки в дополнительных единицах измерения (м3, пары, шт.), регионе-участнике торговли.

Единица наблюдения в датасете – сгруппированный из таможенных деклараций объём импорта или экспорта по десятизначному коду товара между регионом России и страной за месяц.

Всего набор содержит 26 392 291 наблюдений (строк) по 10 атрибутам (столбцам).

[Источник](https://data-in.ru/data-catalog/datasets/201/)

#ЗАГРУЗКА ДАННЫХ

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.filterwarnings(action='once')
from sklearn import preprocessing
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from google.colab import data_table

/usr/local/lib/python3.7/dist-packages/yaml/constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(key, collections.Hashable):
/usr/local/lib/python3.7/dist-packages/dask/array/numpy_compat.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.divide(0.4, 1, casting="unsafe", dtype=np.float),
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/imp

In [ ]:
%%capture
!wget https://www.dropbox.com/s/8eu23mfggeis4hj/data.csv
!wget https://www.dropbox.com/s/z4zqa7l8hb3rd3l/meta.CSV
!wget https://www.dropbox.com/s/okktaz65pl7xqfa/structure.CSV

In [ ]:
data=pd.read_csv('data.csv', engine='c', sep=';', encoding='utf8', decimal=',') #база данных, меняем кодировку, разделитель данных ';', разделитель в числах ','

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
meta=pd.read_csv('meta.CSV') #мета данные

In [ ]:
structure_d=pd.read_csv('structure.CSV', encoding='cp1251', sep=';') #структура данных

In [ ]:
data.columns

Index(['direction', 'month', 'country', 'code', 'measure', 'value', 'netto',
       'quantity', 'region', 'district'],
      dtype='object')

In [ ]:
meta

,property,value
0,standardversion,http://opendata.gosmonitor.ru/standard/3.0
1,identifier,ANO CPUR/АНО ЦПУР
2,title,Экспорт и импорт российских регионов: таможенн...
3,description,Данные Федеральной таможенной службы по экспор...
4,creator,АНО ЦПУР - Центр перспективных управленческих ...
5,created,20220324
6,modified,20220324
7,subject,NaN
8,format,CSV
9,provenance,NaN


In [ ]:
structure_d

,field name,english description,russian description,measure,okei_code,format
0,direction,Direction: export (ЭК) or import (ИМ),Направление поставок: экспорт (ЭК) или импорт ...,NaN,NaN,string
1,month,Delivery year and month,Месяц и год осуществления поставки,NaN,NaN,string
2,country,Destination country code,Кодовое обозначение страны участника торговли ...,NaN,NaN,string
3,code,Commodity Nomenclature of Foreign Economic Act...,Кодовое обозначение товара в соответствии с 10...,NaN,NaN,string
4,measure,Additional measure,Краткое обозначение дополнительных единиц изме...,NaN,NaN,string
5,value,The total volume of goods in deliveries accord...,Суммарный объём товаров в поставках по таможен...,Доллар США,840.0,numeric
6,netto,Total net weight of goods in deliveries accord...,Суммарный вес нетто товаров в поставках по там...,КГ,166.0,numeric
7,quantity,Total quantity of goods according to customs d...,Суммарное количество товаров по таможенным дек...,NaN,NaN,numeric
8,region,Region,Регион,NaN,NaN,string
9,district,Federal district,Федеральный округ,NaN,NaN,string


In [ ]:
data.head()

,direction,month,country,code,measure,value,netto,quantity,region,district
0,ИМ,01/2016,MY,8482109008,ШТ,443.0,30.0,72.0,40,2
1,ИМ,01/2016,IT,6204695000,ШТ,131.0,1.0,7.0,46,1
2,ИМ,01/2016,CN,9001900009,1,112750.0,18.0,0.0,46,1
3,ИМ,01/2016,BY,8414302004,ШТ,392.0,57.0,8.0,50,6
4,ИМ,01/2016,US,9018509000,1,54349.0,179.0,0.0,40,2


In [ ]:
data.info

<bound method DataFrame.info of          direction    month country        code measure      value    netto  \
0               ИМ  01/2016      MY  8482109008      ШТ     443.00     30.0   
1               ИМ  01/2016      IT  6204695000      ШТ     131.00      1.0   
2               ИМ  01/2016      CN  9001900009       1  112750.00     18.0   
3               ИМ  01/2016      BY  8414302004      ШТ     392.00     57.0   
4               ИМ  01/2016      US  9018509000       1   54349.00    179.0   
...            ...      ...     ...         ...     ...        ...      ...   
26392286        ЭК  12/2021      KZ  0301998500       1   16037.86  47178.0   
26392287        ЭК  12/2021      VN  2403999009       1   50142.86   2400.0   
26392288        ИМ  12/2021      BY  4805932000       1   13311.11  20951.0   
26392289        ЭК  12/2021      BY  3926200000       1    1698.23    555.0   
26392290        ЭК  12/2021      KZ  0301930000       1    9564.11  19000.0   

          quantity 

#***РАЗДЕЛ 1***

##ИМПОРТ (ОБЩИЕ ДАННЫЕ)

In [ ]:
im_omsk=data[(data['region'] == 52) & (data['direction'] == 'ИМ')] #в таможенной классификации Омская область имеет код "52000 - ОМСКАЯ ОБЛАСТЬ", выборка по критерию "ИМ" (ИМпорт)
im_omsk

,direction,month,country,code,measure,value,netto,quantity,region,district
1651,ИМ,01/2016,CN,8413910008,1,1015.00,150.00,0.0,52,6
1926,ИМ,01/2016,BY,6108220000,ШТ,496.00,6.00,198.0,52,6
2447,ИМ,01/2016,JP,8407330000,ШТ,516.00,344.00,2.0,52,6
2972,ИМ,01/2016,JP,5705003000,М2,980.00,140.00,34.0,52,6
3154,ИМ,01/2016,JP,8512909009,1,2000.00,80.00,0.0,52,6
...,...,...,...,...,...,...,...,...,...,...
23400053,ИМ,12/2021,BR,0901110001,1,78688.22,19296.80,0.0,52,6
23400056,ИМ,12/2021,KZ,1905329900,1,4748.73,2602.32,0.0,52,6
23400057,ИМ,12/2021,LK,0902400000,1,75910.40,20786.00,0.0,52,6
23400066,ИМ,12/2021,KZ,1905319100,1,564.48,408.00,0.0,52,6


###СТРАНЫ

In [ ]:
im_omsk_country = im_omsk.groupby(['country'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_country = im_omsk_country.sort_values('value', axis=0, ascending=False)
im_omsk_country = im_omsk_country.head(10)
im_omsk_country.round({'value': 1, 'netto': 1})

,country,value,netto
53,KZ,641114174.5,1.250177e+10
19,CN,338276721.2,1.180787e+08
23,DE,196011346.1,3.820154e+07
46,IT,172924755.9,1.040633e+07
14,BY,132272143.8,9.269416e+07
100,US,96722826.9,5.789145e+06
15,CA,71473379.3,6.718992e+06
7,AZ,64482833.7,1.247597e+08
102,UZ,57211564.8,9.355030e+06
71,NL,45805641.8,4.224248e+06


In [ ]:
im_omsk_country.sum()

country    KZCNDEITBYUSCAAZUZNL
value            1816295388.125
netto        12911999915.441238
dtype: object

In [ ]:
im_omsk_country['country'].replace(['KZ'],'КАЗАХСТАН',inplace=True)
im_omsk_country['country'].replace(['CN'],'КИТАЙ',inplace=True)
im_omsk_country['country'].replace(['DE'],'ГЕРМАНИЯ ',inplace=True)
im_omsk_country['country'].replace(['IT'],'ИТАЛИЯ',inplace=True)
im_omsk_country['country'].replace(['BY'],'БЕЛАРУСЬ',inplace=True)
im_omsk_country['country'].replace(['US'],'СОЕДИНЕННЫЕ ШТАТЫ',inplace=True)
im_omsk_country['country'].replace(['CA'],'КАНАДА',inplace=True)
im_omsk_country['country'].replace(['AZ'],'АЗЕРБАЙДЖАН',inplace=True)
im_omsk_country['country'].replace(['UZ'],'УЗБЕКИСТАН',inplace=True)
im_omsk_country['country'].replace(['NL'],'НИДЕРЛАНДЫ',inplace=True)

In [ ]:
labels = im_omsk_country.country
fig = go.Figure()
pull=0.03
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=im_omsk_country.value, name='стоимость ($)', pull=pull, hole=0.7), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=im_omsk_country.netto, name='масса (кг)', pull=pull, hole=0.7, rotation=80), 1, 2)

fig.update_traces(hole=.5, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="СТРАНЫ - ИМПОРТЕРЫ ТОВАРОВ 2016-2021 (ТОП-10)",
    title_x = 0.45,
    annotations=[dict(text='СТОИМОСТЬ', x=0.15, y=0.5, font_size=20, showarrow=False),
                 dict(text='МАССА', x=0.81, y=0.5, font_size=20, showarrow=False)])
fig.show()

###КОД ТНВЭД

In [ ]:
im_omsk_code = im_omsk.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_code = im_omsk_code.sort_values('value', axis=0, ascending=False)
im_omsk_code = im_omsk_code.head(10)
im_omsk_code.round({'value': 1, 'netto': 1})

,code,value,netto
3406,8482500009,185727719.7,6.662665e+07
1020,2701190000,87701188.8,8.542424e+09
1432,3815199000,63522462.2,2.722064e+06
1040,2711140001,61332569.8,1.229246e+08
2995,8419500000,45949037.2,3.976836e+06
2998,8419899890,42255214.2,1.995988e+06
2961,8414805900,40893390.0,1.456723e+06
3117,8432900000,37123895.1,6.912681e+06
481,0401209109,33058812.1,7.214590e+07
1039,2711131000,28661528.9,8.726718e+07


In [ ]:
#расшифровка кодов
im_omsk_code['code'].replace(['8482500009'],'ПОДШИПНИКИ',inplace=True)
im_omsk_code['code'].replace(['2701190000'],'УГОЛЬ КАМЕННЫЙ',inplace=True)
im_omsk_code['code'].replace(['3815199000'],'КАТАЛИЗАТОРЫ',inplace=True)
im_omsk_code['code'].replace(['2711140001'],'СМЕСЬ, СОДЕРЖАЩАЯ УГЛЕВОДОРОДЫ',inplace=True)
im_omsk_code['code'].replace(['8419500000'],'ТЕПЛООБМЕННИКИ',inplace=True)
im_omsk_code['code'].replace(['8419899890'],'ПРОЧИЕ МАШИНЫ, АГРЕГАТЫ',inplace=True)
im_omsk_code['code'].replace(['8414805900'],'КОМПРЕССОРЫ',inplace=True)
im_omsk_code['code'].replace(['8432900000'],'ЧАСТИ МАШИН СЕЛЬСКОХОЗЯЙСТВЕННЫХ',inplace=True)
im_omsk_code['code'].replace(['0401209109'],'МОЛОКО И СЛИВКИ',inplace=True)
im_omsk_code['code'].replace(['2711131000'],'СЖИЖЕННЫЕ БУТАНЫ',inplace=True)

In [ ]:
labels = im_omsk_code.code
fig = go.Figure()
pull=0.03
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=im_omsk_code.value, name='стоимость ($)', pull=pull, hole=0.7), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=im_omsk_code.netto, name='масса (кг)', pull=pull, hole=0.7, rotation=250), 1, 2)

fig.update_traces(hole=.5, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="ИМПОРТ ТОВАРОВ ТН ВЭД 2016-2021 (ТОП-10)",
    title_x = 0.45,
    annotations=[dict(text='СТОИМОСТЬ', x=0.148, y=0.5, font_size=20, showarrow=False),
                 dict(text='МАССА', x=0.823, y=0.5, font_size=20, showarrow=False)])
fig.show()

##ЭКСПОРТ (ОБЩИЕ ДАННЫЕ)

In [ ]:
ex_omsk=data[(data['region'] == 52) & (data['direction'] == 'ЭК')] #в таможенной классификации Омская область имеет код "52000 - ОМСКАЯ ОБЛАСТЬ", выборка по критерию "ЭК" (ЭКспорт)
ex_omsk

,direction,month,country,code,measure,value,netto,quantity,region,district
2448780,ЭК,01/2016,KZ,8539390000,ШТ,5290.00,74.000,1650.0,52,6
2448784,ЭК,01/2016,KZ,8467299000,ШТ,896.00,18.000,5.0,52,6
2448826,ЭК,01/2016,KZ,2007993909,1,266.00,283.000,0.0,52,6
2448827,ЭК,01/2016,KZ,6912005000,1,26.00,16.000,0.0,52,6
2448950,ЭК,01/2016,KZ,8440104000,ШТ,279.00,23.000,4.0,52,6
...,...,...,...,...,...,...,...,...,...,...
23400083,ЭК,12/2021,KZ,1604200500,1,715.37,210.000,0.0,52,6
23400084,ЭК,12/2021,BY,0904120000,1,421.75,57.900,0.0,52,6
23400085,ЭК,12/2021,KZ,2101110016,1,21.00,1.240,0.0,52,6
23400086,ЭК,12/2021,UZ,1806901900,1,59960.00,53548.450,0.0,52,6


###СТРАНЫ

In [ ]:
ex_omsk_country = ex_omsk.groupby(['country'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
ex_omsk_country = ex_omsk_country.sort_values('value', axis=0, ascending=False)
ex_omsk_country = ex_omsk_country.head(10)
ex_omsk_country.round({'value': 1, 'netto': 1})

,country,value,netto
57,KZ,1.309180e+09,1.526023e+09
20,CN,4.029126e+08,6.068488e+08
79,PL,2.708014e+08,3.997406e+08
95,TR,2.510745e+08,3.712117e+08
86,SA,2.466717e+08,9.329020e+06
15,BY,2.358055e+08,2.542881e+08
43,HU,1.882940e+08,2.479084e+08
25,DE,1.805685e+08,2.071537e+08
72,NL,1.753583e+08,2.789440e+08
60,LT,1.160833e+08,1.198918e+08


In [ ]:
#расшифровка стран
ex_omsk_country['country'].replace(['KZ'],'КАЗАХСТАН',inplace=True)
ex_omsk_country['country'].replace(['CN'],'КИТАЙ',inplace=True)
ex_omsk_country['country'].replace(['PL'],'ПОЛЬША ',inplace=True)
ex_omsk_country['country'].replace(['TR'],'ТУРЦИЯ ',inplace=True)
ex_omsk_country['country'].replace(['SA'],'САУДОВСКАЯ АРАВИЯ',inplace=True)
ex_omsk_country['country'].replace(['BY'],'БЕЛАРУСЬ',inplace=True)
ex_omsk_country['country'].replace(['HU'],'ВЕНГРИЯ',inplace=True)
ex_omsk_country['country'].replace(['DE'],'ГЕРМАНИЯ',inplace=True)
ex_omsk_country['country'].replace(['NL'],'НИДЕРЛАНДЫ',inplace=True)
ex_omsk_country['country'].replace(['LT'],'ЛИТВА',inplace=True)

In [ ]:
labels = ex_omsk_country.country
fig = go.Figure()
pull=0.03
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=ex_omsk_country.value, name='стоимость ($)', pull=pull, hole=0.7), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=ex_omsk_country.netto, name='масса (кг)', pull=pull, hole=0.7), 1, 2)

fig.update_traces(hole=.5, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="СТРАНЫ - ЭКСПОРТЕРЫ ТОВАРОВ 2016-2021 (ТОП-10, $)",
    title_x = 0.45,
    annotations=[dict(text='СТОИМОСТЬ', x=0.156, y=0.5, font_size=20, showarrow=False),
                 dict(text='МАССА', x=0.81, y=0.5, font_size=20, showarrow=False)])
fig.show()

###КОД ТНВЭД

In [ ]:
ex_omsk_code = ex_omsk.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
ex_omsk_code = ex_omsk_code.sort_values('value', axis=0, ascending=False)
ex_omsk_code = ex_omsk_code.head(10)
ex_omsk_code.round({'value': 1, 'netto': 1})

,code,value,netto
1286,2803000000,869945648.8,1.064050e+09
6335,XXXXXXXXXX,305165676.4,1.449179e+08
1937,3902100000,296199846.2,2.749815e+08
487,1001990000,196539502.2,9.172193e+08
1453,2909199000,182664444.1,2.973512e+08
2132,4002199000,179961094.7,1.418305e+08
592,1204009000,141568543.9,2.887296e+08
1409,2901100001,138643998.7,3.351210e+08
2175,4011209000,107103882.9,5.673675e+07
670,1514119009,101371709.2,1.072489e+08


In [ ]:
#расшифровка кодов
ex_omsk_code['code'].replace(['2803000000'],'УГЛЕРОД ',inplace=True)
ex_omsk_code['code'].replace(['XXXXXXXXXX'],'XXXXXXXXXX',inplace=True)
ex_omsk_code['code'].replace(['3902100000'],'ПОЛИПРОПИЛЕН ',inplace=True)
ex_omsk_code['code'].replace(['1001990000'],'ПШЕНИЦА ',inplace=True)
ex_omsk_code['code'].replace(['2909199000'],'ЭФИРЫ ПРОСТЫЕ',inplace=True)
ex_omsk_code['code'].replace(['4002199000'],'ПРОЧИЙ КАУЧУК',inplace=True)
ex_omsk_code['code'].replace(['1204009000'],'ПРОЧИЕ СЕМЕНА ЛЬНА',inplace=True)
ex_omsk_code['code'].replace(['2901100001'],'УГЛЕВОДОРОДЫ',inplace=True)
ex_omsk_code['code'].replace(['4011209000'],'ШИНЫ И ПОКРЫШКИ',inplace=True)
ex_omsk_code['code'].replace(['1514119009'],'ПРОЧЕЕ МАСЛО РАПСОВОЕ',inplace=True)

In [ ]:
labels = ex_omsk_code.code
fig = go.Figure()
pull=0.03
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=ex_omsk_code.value, name='стоимость ($)', pull=pull, hole=0.7), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=ex_omsk_code.netto, name='масса (кг)', pull=pull, hole=0.7), 1, 2)

fig.update_traces(hole=.5, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="КОДЫ ЭКСПОРТА ТОВАРОВ ТН ВЭД 2016-2021 (ТОП-10, $)",
    title_x = 0.45,
    annotations=[dict(text='СТОИМОСТЬ', x=0.156, y=0.5, font_size=20, showarrow=False),
                 dict(text='МАССА', x=0.81, y=0.5, font_size=20, showarrow=False)])
fig.show()

#***РАЗДЕЛ 2***

##ИМПОРТ (ВЫБОРКА)

In [ ]:
#выборка по странам - Казахстан (KZ), Узбекистан (UZ), Киргизия (KG), Иран (IR), Индия (IN), Пакистан (PK), Китай (CN), Монголия (MN)
#в таможенной классификации Омская область имеет код "52000 - ОМСКАЯ ОБЛАСТЬ", выборка по критерию "ИМ" (ИМпорт)

im_omsk_KZ=data[(data['region'] == 52) & (data['direction'] == 'ИМ') & (data['country'] == 'KZ')]
im_omsk_KZ = im_omsk_KZ.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_KZ = im_omsk_KZ.sort_values('value', axis=0, ascending=False)
im_omsk_KZ = im_omsk_KZ.head(20)
im_omsk_KZ.round({'value': 1, 'netto': 1})

,code,value,netto
1119,8482500009,185318719.7,6.662340e+07
515,2701190000,87701188.8,8.542424e+09
131,0401209109,33058812.1,7.214590e+07
527,2711131000,28661528.9,8.726718e+07
890,7602009000,20100627.1,1.611362e+07
129,0401201109,18799460.0,4.440448e+07
519,2707999100,15679542.0,9.892420e+07
96,0207119009,12948450.0,8.542645e+06
284,1001190000,11230296.5,4.958787e+07
516,2702100000,10600266.8,1.156880e+09


In [ ]:
#расшифровка кодов
im_omsk_KZ['code'].replace(['8482500009'],'ПОДШИПНИКИ ',inplace=True)
im_omsk_KZ['code'].replace(['2701190000'],'УГОЛЬ КАМЕННЫЙ',inplace=True)
im_omsk_KZ['code'].replace(['0401209109'],'МОЛОКО И СЛИВКИ',inplace=True)
im_omsk_KZ['code'].replace(['2711131000'],'СЖИЖЕННЫЕ БУТАНЫ',inplace=True)
im_omsk_KZ['code'].replace(['7602009000'],'АЛЮМИНИЙ И ИЗДЕЛИЯ ИЗ НЕГО',inplace=True)
im_omsk_KZ['code'].replace(['0401201109'],'МОЛОКО И СЛИВКИ. ПРОЧИЕ',inplace=True)
im_omsk_KZ['code'].replace(['2707999100'],'ПРОЧИЕ МАСЛА',inplace=True)
im_omsk_KZ['code'].replace(['0207119009'],'МЯСО ПТИЦЫ. ПРОЧИЕ',inplace=True)
im_omsk_KZ['code'].replace(['1001190000'],'ПРОЧАЯ, ПШЕНИЦА ТВЕРДАЯ',inplace=True)
im_omsk_KZ['code'].replace(['2702100000'],'ЛИГНИТ. БУРЫЙ УГОЛЬ',inplace=True)
im_omsk_KZ['code'].replace(['2902700000'],'КУМОЛ',inplace=True)
im_omsk_KZ['code'].replace(['2517101000'],'ГАЛЬКА, ГРАВИЙ, ЩЕБЕНЬ',inplace=True)
im_omsk_KZ['code'].replace(['0401209900'],'ПРОЧЕЕ МОЛОКО И СЛИВКИ',inplace=True)
im_omsk_KZ['code'].replace(['1704907100'],'ЛЕДЕНЦОВАЯ КАРАМЕЛЬ',inplace=True)
im_omsk_KZ['code'].replace(['8482990000'],'СТЯЖНАЯ ВТУЛКА ПОДШИПНИКА',inplace=True)
im_omsk_KZ['code'].replace(['1517109000'],'ПРОЧИЙ МАРГАРИН',inplace=True)
im_omsk_KZ['code'].replace(['0406900100'],'ПРОЧИЕ СЫРЫ',inplace=True)
im_omsk_KZ['code'].replace(['7210122000'],'БЕЛАЯ ЖЕСТЬ',inplace=True)
im_omsk_KZ['code'].replace(['2706000000'],'СМОЛЫ КАМЕННОУГОЛЬНЫЕ',inplace=True)
im_omsk_KZ['code'].replace(['0201300004'],'МЯСО КРС',inplace=True)

In [ ]:
fig = px.scatter(im_omsk_KZ, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Импорт товаров из Казахстана 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
im_omsk_UZ=data[(data['region'] == 52) & (data['direction'] == 'ИМ') & (data['country'] == 'UZ')]
im_omsk_UZ = im_omsk_UZ.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_UZ = im_omsk_UZ.sort_values('value', axis=0, ascending=False)
im_omsk_UZ = im_omsk_UZ.head(20)
im_omsk_UZ.round({'value': 1, 'netto': 1})

,code,value,netto
83,6109100000,12391546.0,1236715.2
111,8605000008,7948289.7,969800.0
75,6104620000,6445462.9,719463.0
73,6104420000,3776726.1,368791.9
81,6108310000,3545708.6,488817.7
82,6108910000,3064872.8,342116.1
86,6110209900,2547859.4,223294.4
104,6302600000,2347824.8,467273.3
78,6107110000,2247279.2,228439.2
70,6103420001,2170154.8,241083.4


In [ ]:
im_omsk_UZ['code'].replace(['6109100000'],'МАЙКИ, ФУФАЙКИ С РУКАВАМИ',inplace=True)
im_omsk_UZ['code'].replace(['8605000008'],'ПРОЧИЕ ВАГОНЫ ЖЕЛЕЗНОДОР',inplace=True)
im_omsk_UZ['code'].replace(['6104620000'],'БРЮКИ, КОМБИНЕЗОНЫ',inplace=True)
im_omsk_UZ['code'].replace(['6104420000'],'ПЛАТЬЯ ТРИКОТАЖНЫЕ',inplace=True)
im_omsk_UZ['code'].replace(['6108310000'],'НОЧНЫЕ СОРОЧКИ И ПИЖАМЫ',inplace=True)
im_omsk_UZ['code'].replace(['6108910000'],'ПРОЧИЕ НОЧНЫЕ СОРОЧКИ И ПИЖАМЫ',inplace=True)
im_omsk_UZ['code'].replace(['6110209900'],'ПРОЧИЕ СВИТЕРА, ПУЛОВЕРЫ, ДЖЕМПЕРЫ',inplace=True)
im_omsk_UZ['code'].replace(['6302600000'],'БЕЛЬЕ ТУАЛЕТНОЕ И КУХОННОЕ',inplace=True)
im_omsk_UZ['code'].replace(['6107110000'],'ТРУСЫ И КАЛЬСОНЫ МУЖСКИЕ',inplace=True)
im_omsk_UZ['code'].replace(['6103420001'],'БРЮКИ И БРИДЖЫ МУЖСКИЕ',inplace=True)
im_omsk_UZ['code'].replace(['6110209100'],'ПРОЧИЕ СВИТЕРА, ПУЛОВЕРЫ, ДЖЕМПЕРЫ',inplace=True)
im_omsk_UZ['code'].replace(['6111209000'],'ПРОЧАЯ ДЕТСКАЯ ОДЕЖДА И ПРИНАДЛЕЖНОСТИ',inplace=True)
im_omsk_UZ['code'].replace(['5503200000'],'ВОЛОКНА ПОЛИЭФИРНЫЕ',inplace=True)
im_omsk_UZ['code'].replace(['6103420009'],'КОМБИНЕЗОНЫ С НАГРУДНИКАМИ И ЛЯМКАМИ',inplace=True)
im_omsk_UZ['code'].replace(['6005210000'],'ПОЛОТНА ОСНОВОВЯЗАННЫЕ',inplace=True)
im_omsk_UZ['code'].replace(['5607909000'],'ПРОЧИЕ БЕЧЕВКИ, ВЕРЕВКИ',inplace=True)
im_omsk_UZ['code'].replace(['6107210000'],'НОЧНЫЕ СОРОЧКИ И ПИЖАМЫ МУЖСКИЕ',inplace=True)
im_omsk_UZ['code'].replace(['6108210000'],'ТРУСЫ И ПАНТАЛОНЫ ТРИКОТАЖНЫЕ',inplace=True)
im_omsk_UZ['code'].replace(['6110201000'],'СВИТЕРЫ, ПУЛОВЕРЫ, КАРДИГАНЫ',inplace=True)
im_omsk_UZ['code'].replace(['6112110000'],'КОСТЮМЫ СПОРТИВНЫЕ, ТРИКОТАЖНЫЕ',inplace=True)

In [ ]:
fig = px.scatter(im_omsk_UZ, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Импорт товаров из Узбекистана 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
im_omsk_KG=data[(data['region'] == 52) & (data['direction'] == 'ИМ') & (data['country'] == 'KG')]
im_omsk_KG = im_omsk_KG.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_KG = im_omsk_KG.sort_values('value', axis=0, ascending=False)
im_omsk_KG = im_omsk_KG.head(20)
im_omsk_KG.round({'value': 1, 'netto': 1})

,code,value,netto
4,0303141000,5436866.0,1235644.9
14,0406900100,1507520.5,704000.0
15,0406908600,1265036.8,620967.0
0,0302111000,1250007.4,281242.2
94,7404001000,1203887.0,213384.0
86,7005293500,1028800.6,2814320.0
5,0303142000,1026895.0,224495.9
6,0303149000,679556.3,152487.0
1,0302112000,585290.8,120903.8
11,0405101100,545613.2,136500.0


In [ ]:
im_omsk_KG['code'].replace(['0303141000'],'ФОРЕЛЬ ВИДА ONCORHYNCHUS APACHE',inplace=True)
im_omsk_KG['code'].replace(['0406900100'],'ПРОЧИЕ СЫРЫ',inplace=True)
im_omsk_KG['code'].replace(['0406908600'],'ПРОЧИЕ СЫРЫ',inplace=True)
im_omsk_KG['code'].replace(['0302111000'],'ФОРЕЛЬ ВИДОВ ONCORHYNCHUS APACHE',inplace=True)
im_omsk_KG['code'].replace(['7404001000'],'ОТХОДЫ РАФИНИРОВАННОЙ МЕДИ',inplace=True)
im_omsk_KG['code'].replace(['7005293500'],'СТЕКЛО НЕАРМИРОВАННОЕ ПРОЧЕЕ',inplace=True)
im_omsk_KG['code'].replace(['0303142000'],'ФОРЕЛЬ ВИДА ONCORHYNCHUS MYKISS',inplace=True)
im_omsk_KG['code'].replace(['0303149000'],'ПРОЧАЯ ФОРЕЛЬ',inplace=True)
im_omsk_KG['code'].replace(['0302112000'],'ФОРЕЛЬ ВИДОВ ONCORHYNCHUS MYKISS',inplace=True)
im_omsk_KG['code'].replace(['0405101100'],'НАТУРАЛЬНОЕ СЛИВОЧНОЕ МАСЛО',inplace=True)
im_omsk_KG['code'].replace(['0704909000'],'ПРОЧИЕ АНАЛОГИЧНЫЕ СЪЕДОБНЫЕ ОВОЩИ',inplace=True)
im_omsk_KG['code'].replace(['0405101900'],'НАТУРАЛЬНОЕ СЛИВОЧНОЕ МАСЛО',inplace=True)
im_omsk_KG['code'].replace(['7602009000'],'ЛОМ АЛЮМИНИЕВЫЙ',inplace=True)
im_omsk_KG['code'].replace(['6104610001'],'БРЮКИ, КОМБИНЕЗОНЫ С НАГРУДНИКАМИ',inplace=True)
im_omsk_KG['code'].replace(['0706100001'],'МОРКОВЬ СВЕЖАЯ ИЛИ ОХЛАЖДЕННАЯ',inplace=True)
im_omsk_KG['code'].replace(['6104620000'],'БРЮКИ, КОМБИНЕЗОНЫ С НАГРУДНИКАМ',inplace=True)
im_omsk_KG['code'].replace(['6206400000'],'БЛУЗКИ, БЛУЗЫ И БЛУЗОНЫ ЖЕСКИЕ',inplace=True)
im_omsk_KG['code'].replace(['8402191009'],'КОТЛЫ ПАРОВЫЕ. ПРОЧИЕ',inplace=True)
im_omsk_KG['code'].replace(['6212109000'],'ПРОЧИЕ БЮСТГАЛТЕРЫ ТРИКОТАЖНЫЕ',inplace=True)
im_omsk_KG['code'].replace(['0813409500'],'ПРОЧИЕ ПЛОДЫ СУШЕНЫЕ',inplace=True)

In [ ]:
fig = px.scatter(im_omsk_KG, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Импорт товаров из Киргизии 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
im_omsk_IR=data[(data['region'] == 52) & (data['direction'] == 'ИМ') & (data['country'] == 'IR')]
im_omsk_IR = im_omsk_IR.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_IR = im_omsk_IR.sort_values('value', axis=0, ascending=False)
im_omsk_IR = im_omsk_IR.head(20)
im_omsk_IR.round({'value': 1, 'netto': 1})

,code,value,netto
3,2901240001,3731197.4,3375060.0
2,2711140001,2446354.0,4018898.0
7,6802991000,54027.0,104040.0
12,XXXXXXXXXX,52942.3,67898.0
9,7326191000,32259.3,18803.8
0,0702000001,22400.0,16000.0
4,3802900000,4415.3,20000.0
1,1211908509,3294.0,150.0
8,7308300000,1672.9,840.0
5,4911101009,270.6,138.5


In [ ]:
im_omsk_IR['code'].replace(['2901240001'],'БУТА-1',inplace=True)
im_omsk_IR['code'].replace(['2711140001'],'СМЕСЬ, СОДЕРЖАЩАЯ УГЛЕРОДЫ С4',inplace=True)
im_omsk_IR['code'].replace(['6802991000'],'ПРОЧИЕ КАМНИ ПОЛИРОВАННЫЕ',inplace=True)
im_omsk_IR['code'].replace(['XXXXXXXXXX'],'XXXXXXXXXX',inplace=True)
im_omsk_IR['code'].replace(['7326191000'],'ПРОЧИЕ ИЗДЕЛИЯ ИЗ ЧЕРНЫХ МЕТАЛЛОВ КОВАНЫЕ',inplace=True)
im_omsk_IR['code'].replace(['0702000001'],'ТОМАТЫ СВЕЖИЕ ИЛИ ОХЛАЖДЕННЫЕ',inplace=True)
im_omsk_IR['code'].replace(['3802900000'],'ПРОЧИЕ ПРОДУКТЫ МИНЕРАЛЬНЫЕ ПРИРОДНЫЕ АКТИВИРОВАННЫЕ',inplace=True)
im_omsk_IR['code'].replace(['1211908509'],'ГРИБ БЕРЕЗОВЫЙ (ЧАГА)',inplace=True)
im_omsk_IR['code'].replace(['7308300000'],'ДВЕРИ, ОКНА И ИХ РАМЫ',inplace=True)
im_omsk_IR['code'].replace(['4911101009'],'ТОВАРНЫЕ КАТАЛОГИ ПРОЧИЕ',inplace=True)
im_omsk_IR['code'].replace(['8306290001'],'СТАТУЭТКИ И ДРУГИЕ УКРАШЕНИЯ ИЗ МЕДИ',inplace=True)
im_omsk_IR['code'].replace(['4911990000'],'ПРОЧАЯ ПЕЧАТНАЯ ПРОДУКЦИЯ',inplace=True)
im_omsk_IR['code'].replace(['9609901000'],'ПАСТЕЛИ И УГОЛЬНЫЕ КАРАНДАШИ',inplace=True)

In [ ]:
fig = px.scatter(im_omsk_IR, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Импорт товаров из Ирана 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
im_omsk_IN=data[(data['region'] == 52) & (data['direction'] == 'ИМ') & (data['country'] == 'IN')]
im_omsk_IN = im_omsk_IN.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_IN = im_omsk_IN.sort_values('value', axis=0, ascending=False)
im_omsk_IN = im_omsk_IN.head(20)
im_omsk_IN.round({'value': 1, 'netto': 1})

,code,value,netto
216,8419899890,5055836.4,130462.5
214,8414805900,3723217.4,57800.0
150,4011700000,845042.4,269048.0
10,0902400000,764725.0,395122.0
75,2403199000,648386.0,252700.0
156,4901100000,644648.0,14.6
63,2101110011,540099.4,99450.9
230,8477400000,520779.6,47360.0
137,3823110000,519557.4,445850.0
221,8433900000,479827.1,101359.9


In [ ]:
im_omsk_IN['code'].replace(['8419899890'],'ПРОЧИЕ МАШИНЫ, АГРЕГАТЫ И ОБОРУДОВАНИЕ',inplace=True)
im_omsk_IN['code'].replace(['8414805900'],'КОМПРЕССОРЫ ОБЪЕМНЫЕ ВОЗВРАТНО-ПОСТУПАТЕЛЬНЫЕ',inplace=True)
im_omsk_IN['code'].replace(['4011700000'],'ШИНЫ И ПОКРЫШКИ ПНЕВМАТИЧЕСКИЕ РЕЗИНОВЫЕ',inplace=True)
im_omsk_IN['code'].replace(['0902400000'],'ЧАЙ ЧЕРНЫЙ (ФЕРМЕНТИРОВАННЫЙ)',inplace=True)
im_omsk_IN['code'].replace(['2403199000'],'ПРОЧИЙ КУРИТЕЛЬНЫЙ ТАБАК',inplace=True)
im_omsk_IN['code'].replace(['4901100000'],'КНИГИ, БРОШЮРЫ, ЛИСТОВКИ',inplace=True)
im_omsk_IN['code'].replace(['2101110011'],'КОФЕ РАСТВОРИМЫЙ ПОРОШКООБРАЗНЫЙ',inplace=True)
im_omsk_IN['code'].replace(['8477400000'],'МАШИНЫ ДЛЯ ВАКУУМНОГО ЛИТЬЯ',inplace=True)
im_omsk_IN['code'].replace(['3823110000'],'ПРОМЫШЛЕННАЯ СТЕАРИНОВАЯ КИСЛОТА',inplace=True)
im_omsk_IN['code'].replace(['8433900000'],'ЧАСТИ МАШИН ИЛИ МЕХАНИЗМОВ ДЛЯ УБОРКИ ИЛИ ОБМОЛОТА С/Х КУЛЬТУР',inplace=True)
im_omsk_IN['code'].replace(['8477200000'],'ЭКСТРУДЕРЫ ',inplace=True)
im_omsk_IN['code'].replace(['3824999609'],'ПРОЧИЕ ПРОДУКТЫ И ПРЕПАРАТЫ ХИМИЧЕСКИЕ (ЛЕКАРСТВА)',inplace=True)
im_omsk_IN['code'].replace(['7326909807'],'ПРОЧИЕ ИЗДЕЛИЯ ПРОЧИЕ ИЗ ЧЕРНЫХ МЕТАЛЛОВ',inplace=True)
im_omsk_IN['code'].replace(['2101110012'],'ПРОЧИЙ КОФЕ РАСТВОРИМЫЙ ПОРОШКООБРАЗНЫЙ',inplace=True)
im_omsk_IN['code'].replace(['4016999708'],'ИЗДЕЛИЯ ИЗ ВУЛКАНИЗОВАННОЙ РЕЗИНЫ',inplace=True)
im_omsk_IN['code'].replace(['3802900000'],'ПРОЧИЕ ПРОДУКТЫ МИНЕРАЛЬНЫЕ ПРИРОДНЫЕ АКТИВИРОВАННЫЕ',inplace=True)
im_omsk_IN['code'].replace(['2831100000'],'ДИТИОНИТЫ И СУЛЬФОКСИЛАТЫ НАТРИЯ',inplace=True)
im_omsk_IN['code'].replace(['6802931000'],'ГРАНИТ ПОЛИРОВАННЫЙ, ДЕКОРИРОВАННЫЙ',inplace=True)
im_omsk_IN['code'].replace(['2920907000'],'ПРОЧИЕ СОЕДИНЕНИЯ СЛОЖНЫХ ЭФИРОВ ПРОЧИХ НЕОРГАНИЧЕСКИХ КИСЛОТ НЕМЕТАЛЛОВ',inplace=True)
im_omsk_IN['code'].replace(['6909190009'],'ПРОЧИЕ ИЗДЕЛИЯ КЕРАМИЧЕСКИЕ ДЛЯ ЛАБОРАТОРНЫХ, ХИМИЧЕСКИХ ИЛИ ТЕХНИЧЕСКИХ ЦЕЛЕЙ',inplace=True)

In [ ]:
fig = px.scatter(im_omsk_IN, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Импорт товаров из Индии 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
im_omsk_PK=data[(data['region'] == 52) & (data['direction'] == 'ИМ') & (data['country'] == 'PK')]
im_omsk_PK = im_omsk_PK.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_PK = im_omsk_PK.sort_values('value', axis=0, ascending=False)
im_omsk_PK = im_omsk_PK.head(20)
im_omsk_PK.round({'value': 1, 'netto': 1})

,code,value,netto
1,2501009190,103576.1,228722.2
17,6302600000,99292.3,18414.6
25,9506620000,35396.9,1490.0
2,2529210000,18011.5,52016.0
0,2501009190,11376.0,22860.0
18,6802999000,7511.9,14302.6
9,6109902000,5448.6,194.1
20,8203200001,1488.6,21.8
12,6110309100,1295.3,30.7
14,6116108000,1242.2,42.4


In [ ]:
im_omsk_PK['code'].replace(['2501009190'],'ПРОЧАЯ СОЛЬ, ПРИГОДНАЯ ДЛЯ УПОТРЕБЛЕНИЯ В ПИЩУ',inplace=True)
im_omsk_PK['code'].replace(['6302600000'],'БЕЛЬЕ ТУАЛЕТНОЕ И КУХОННОЕ',inplace=True)
im_omsk_PK['code'].replace(['9506620000'],'МЯЧИ НАДУВНЫЕ',inplace=True)
im_omsk_PK['code'].replace(['2529210000'],'СОЛЬ С СОДЕРЖАНИЕМ ФТОРИДА КАЛЬЦИЯ',inplace=True)
im_omsk_PK['code'].replace(['2501009190'],'ПРОЧАЯ СОЛЬ, ПРИГОДНАЯ ДЛЯ УПОТРЕБЛЕНИЯ В ПИЩУ',inplace=True)
im_omsk_PK['code'].replace(['6802999000'],'ИЗДЕЛИЯ ИЗ КАМНЯ, ГИПСА, ЦЕМЕНТА, АСБЕСТА, СЛЮДЫ',inplace=True)
im_omsk_PK['code'].replace(['6109902000'],'МАЙКИ, ФУФАЙКИ С РУКАВАМИ',inplace=True)
im_omsk_PK['code'].replace(['8203200001'],'ПИНЦЕТЫ',inplace=True)
im_omsk_PK['code'].replace(['6110309100'],'ПРОЧИЕ СВИТЕРА, ПУЛОВЕРЫ, ДЖЕМПЕРЫ',inplace=True)
im_omsk_PK['code'].replace(['6116108000'],'ПРОЧИЕ ПЕРЧАТКИ, РУКАВИЦЫ И МИТЕНКИ',inplace=True)
im_omsk_PK['code'].replace(['9506999000'],'ПРОЧИЙ ИНВЕТАРЬ ДЛЯ ЗАНЯТИЯ РАЗЛИЧНЫМИ ВИДАМИ СПОРТА',inplace=True)
im_omsk_PK['code'].replace(['6109909000'],'ПРОЧИЕ МАЙКИ, ФУФАЙКИ С РУКАВАМИ И ПРОЧИЕ НАТЕЛЬНЫЕ',inplace=True)
im_omsk_PK['code'].replace(['6110209100'],'ПРОЧИЕ СВИТЕРА, ПУЛОВЕРЫ, ДЖЕМПЕРЫ, ЖИЛЕТЫ',inplace=True)
im_omsk_PK['code'].replace(['6109100000'],'МАЙКИ, ФУФАЙКИ С РУКАВАМИ И ПРОЧИЕ НАТЕЛЬНЫЕ',inplace=True)
im_omsk_PK['code'].replace(['6201930000'],'ПРОЧИЕ КУРТКИ, ВЕТРОВКИ, ШТОРМОВКИ МУЖСКИЕ',inplace=True)
im_omsk_PK['code'].replace(['7009920000'],'ПРОЧИЕ ЗЕРКАЛА СТЕКЛЯННЫЕ В РАМАХ',inplace=True)
im_omsk_PK['code'].replace(['6105201000'],'РУБАШКИ ТРИКОТАЖНЫЕ, МУЖСКИЕ',inplace=True)
im_omsk_PK['code'].replace(['8213000000'],'НОЖНИЦЫ ПОРТНОВСКИЕ И АНАЛОГИЧНЫЕ',inplace=True)
im_omsk_PK['code'].replace(['6103420001'],'БРЮКИ И БРИДЖЫ, МУЖСКИЕ',inplace=True)
im_omsk_PK['code'].replace(['8205598099'],'СТЕПЛЕР-ПЛАЕР (ИНСТРУМЕНТЫ)',inplace=True)

In [ ]:
fig = px.scatter(im_omsk_PK, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Импорт товаров из Пакистана 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
im_omsk_CN=data[(data['region'] == 52) & (data['direction'] == 'ИМ') & (data['country'] == 'CN')]
im_omsk_CN = im_omsk_CN.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_CN = im_omsk_CN.sort_values('value', axis=0, ascending=False)
im_omsk_CN = im_omsk_CN.head(20)
im_omsk_CN.round({'value': 1, 'netto': 1})

,code,value,netto
1438,8479899708,14730372.3,4214473.8
299,3907610000,11503942.5,11352000.0
1209,8419400009,9638859.5,1533915.7
1462,8481806390,9498780.5,3521610.4
1736,8545110089,9443258.4,884681.8
1210,8419500000,8736493.2,1139898.1
1134,8408105900,6531461.6,1040876.0
386,4011800000,5861863.2,2373205.7
517,5512199000,4996623.4,1390683.5
228,2905120000,4542059.9,2051200.0


In [ ]:
im_omsk_CN['code'].replace(['8479899708'],'ПРОЧИЕ МАШИНЫ И МЕХАНИЗМЫ',inplace=True)
im_omsk_CN['code'].replace(['3907610000'],'ПОЛИЭТИЛЕНТЕРЕФТАЛАТ ',inplace=True)
im_omsk_CN['code'].replace(['8419400009'],'ПРОЧИЕ АППАРАТЫ ДЛЯ ДИСТИЛЛЯЦИИ ИЛИ РЕКТИФИКАЦИИ',inplace=True)
im_omsk_CN['code'].replace(['8481806390'],'ПРОЧИЕ ЗАДВИЖКИ ИЗ СТАЛИ',inplace=True)
im_omsk_CN['code'].replace(['8545110089'],'ЭЛЕКТРОДЫ ПРОЧИЕ, ИСПОЛЬЗУЕМЫЕ В ПЕЧАХ, ПРОЧИЕ.',inplace=True)
im_omsk_CN['code'].replace(['8419500000'],'ТЕПЛООБМЕННИКИ ',inplace=True)
im_omsk_CN['code'].replace(['8408105900'],'ДВИГАТЕЛИ ВНУТРЕННЕГО СГОРАНИЯ ПОРШНЕВЫЕ',inplace=True)
im_omsk_CN['code'].replace(['4011800000'],'ШИНЫ И ПОКРЫШКИ, ДЛЯ ТРАНСПОРТНЫХ СРЕДСТВ И МАШИН',inplace=True)
im_omsk_CN['code'].replace(['5512199000'],'ПРОЧИЕ ТКАНИ, СОДЕРЖАЩИЕ 85 МАС. % ИЛИ БОЛЕЕ ПОЛИЭФИРНЫХ ВОЛОКОН',inplace=True)
im_omsk_CN['code'].replace(['2905120000'],'ИЗОПРОПИЛОВЫЙ СПИРТ (100%).',inplace=True)
im_omsk_CN['code'].replace(['1202420000'],'АРАХИС СВЕЖИЙ ДЛЯ ПРОДОВОЛЬСТВЕННЫХ ЦЕЛЕЙ',inplace=True)
im_omsk_CN['code'].replace(['8502118000'],'УСТАНОВКИ ЭЛЕКТРОГЕНЕРАТОРНЫЕ С ПОРШНЕВЫМ ДВИГАТ. ВНУТР. СГОРАНИЯ',inplace=True)
im_omsk_CN['code'].replace(['6907219009'],'ПЛИТЫ ДЛЯ МОЩЕНИЯ, ПЛИТКИ ОБЛИЦОВОЧНЫЕ ДЛЯ ПОЛОВ',inplace=True)
im_omsk_CN['code'].replace(['8425410000'],'СТАЦИОНАРНЫЕ ГАРАЖНЫЕ ПОДЪЕМНИКИ',inplace=True)
im_omsk_CN['code'].replace(['9018410000'],'БОРМАШИНЫ НА ЕДИНОМ ОСНОВАНИИ С ПРОЧИМ СТОМ. ОБОРУД.',inplace=True)
im_omsk_CN['code'].replace(['7308909809'],'МЕТАЛЛОКОНСТРУКЦИИ ИЗ ЧЕРНЫХ МЕТАЛЛОВ',inplace=True)
im_omsk_CN['code'].replace(['8480710000'],'ФОРМЫ ДЛЯ ЛИТЬЯ РЕЗИНЫ ИЛИ ПЛАСТМАСС',inplace=True)
im_omsk_CN['code'].replace(['5111308001'],'ТКАНИ ПРОЧИЕ, СМЕШАННЫЕ',inplace=True)
im_omsk_CN['code'].replace(['8502120000'],'УСТАНОВКИ ЭЛЕКТРОГЕНЕРАТОРНЫЕ С ПОРШНЕВЫМ ДВИГАТ. ВНУТР. СГОРАНИЯ',inplace=True)
im_omsk_CN['code'].replace(['8483405900'],'ВАРИАТОРЫ СКОРОСТИ, ПРОЧИЕ (КР. КОРОБКИ ПЕРЕДАЧ).',inplace=True)

In [ ]:
fig = px.scatter(im_omsk_CN, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Импорт товаров из Китая 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
im_omsk_MN=data[(data['region'] == 52) & (data['direction'] == 'ИМ') & (data['country'] == 'MN')]
im_omsk_MN = im_omsk_MN.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_MN = im_omsk_MN.sort_values('value', axis=0, ascending=False)
im_omsk_MN = im_omsk_MN.head(20)
im_omsk_MN.round({'value': 1, 'netto': 1})

,code,value,netto
2,2529210000,462854.5,1545178.0
3,2529210001,145893.9,404682.0
8,6110119000,44925.8,182.5
4,2529220000,41238.0,127464.0
1,1207509000,23164.8,66330.0
0,2529210000,15652.0,66606.0
7,6102101000,5288.7,11.3
10,6505009000,1688.5,3.6
9,6116910000,1570.0,4.7
5,2818200000,252.8,0.1


In [ ]:
im_omsk_MN['code'].replace(['2529210000'],'ПЛАВИКОВЫЙ ШПАТ С СОДЕРЖАНИЕМ ФТОРИДА КАЛЬЦИЯ (97%)',inplace=True)
im_omsk_MN['code'].replace(['2529210001'],'ПЛАВИКОВЫЙ ШПАТ, С СОДЕРЖАНИЕМ ФТОРИДА КАЛЬЦИЯ (95%)',inplace=True)
im_omsk_MN['code'].replace(['6110119000'],'ПРОЧИЕ КАРДИГАНЫ, ЖИЛЕТЫ И ИЗДЕЛИЯ ТРИКОТАЖНЫЕ, ДЛЯ ЖЕНЩИН',inplace=True)
im_omsk_MN['code'].replace(['2529220000'],'ПОЛЕВОЙ ШПАТ; ЛЕЙЦИТ; НЕФЕЛИН И НЕФЕЛИНОВЫЙ СИЕНИТ;',inplace=True)
im_omsk_MN['code'].replace(['1207509000'],'ПРОЧИЕ СЕМЕНА ГОРЧИЦЫ',inplace=True)
im_omsk_MN['code'].replace(['2529210000'],'ПЛАВИКОВЫЙ ШПАТ',inplace=True)
im_omsk_MN['code'].replace(['6102101000'],'ПАЛЬТО, ПОЛУПАЛЬТО, НАКИДКИ, ПЛАЩИ И ИЗДЕЛИЯ ЖЕНСКИЕ',inplace=True)
im_omsk_MN['code'].replace(['6505009000'],'ШЛЯПЫ И ПРОЧИЕ ГОЛОВНЫЕ УБОРЫ, ПРОЧИЕ.',inplace=True)
im_omsk_MN['code'].replace(['6116910000'],'ПРОЧИЕ ПЕРЧАТКИ, РУКАВИЦЫ И МИТЕНКИ ТРИКОТАЖНЫЕ ',inplace=True)
im_omsk_MN['code'].replace(['2818200000'],'ОКСИД АЛЮМИНИЯ, ОТЛИЧНЫЙ ОТ ИСКУССТВЕННОГО КОРУНДА.',inplace=True)
im_omsk_MN['code'].replace(['4911990000'],'ПРОЧАЯ ПЕЧАТНАЯ ПРОДУКЦИЯ',inplace=True)

In [ ]:
fig = px.scatter(im_omsk_MN, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Импорт товаров из Монголии 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

##ЭКСПОРТ (ВЫБОРКА)

In [ ]:
#выборка по странам - Казахстан (KZ), Узбекистан (UZ), Киргизия (KG), Иран (IR), Индия (IN), Пакистан (PK), Китай (CN), Монголия (MN)
#в таможенной классификации Омская область имеет код "52000 - ОМСКАЯ ОБЛАСТЬ", выборка по критерию "ИМ" (ИМпорт)

ex_omsk_KZ=data[(data['region'] == 52) & (data['direction'] == 'ЭК') & (data['country'] == 'KZ')]
ex_omsk_KZ = ex_omsk_KZ.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
ex_omsk_KZ = ex_omsk_KZ.sort_values('value', axis=0, ascending=False)
ex_omsk_KZ = ex_omsk_KZ.head(20)
ex_omsk_KZ.round({'value': 1, 'netto': 1})

,code,value,netto
480,1001990000,60649778.3,329443427.4
2124,4011209000,39615010.7,21442851.7
2121,4011100003,35265092.5,14082373.0
3389,7010904300,32774004.6,115918032.7
4537,8432391900,32240400.2,11421203.0
685,1601009900,31998209.5,12649399.9
211,0402991000,26581298.8,19725542.1
4534,8432311900,24828980.4,11324299.0
838,1901909900,22689935.3,18512882.5
5924,9403700008,19515621.0,5094255.0


In [ ]:
ex_omsk_KZ['code'].replace(['1001990000'],'ПРОЧИЕ ПШЕНИЦА И МЕСЛИН',inplace=True)
ex_omsk_KZ['code'].replace(['4011209000'],'ШИНЫ И ПОКРЫШКИ ПНЕВМАТИЧЕСКИЕ РЕЗИНОВЫЕ НОВЫЕ (АВТОБУСЫ)',inplace=True)
ex_omsk_KZ['code'].replace(['4011100003'],'ШИНЫ И ПОКРЫШКИ ПНЕВМАТИЧЕСКИЕ РЕЗИНОВЫЕ НОВЫЕ (ЛЕГКОВЫЕ)',inplace=True)
ex_omsk_KZ['code'].replace(['7010904300'],'БУТЫЛКИ ПРОЧИЕ ДЛЯ ТОВАРОВ ИЗ БЕСЦВЕТНОГО СТЕКЛА 0,33-1 Л.',inplace=True)
ex_omsk_KZ['code'].replace(['8432391900'],'ПРОЧИЕ СЕЯЛКИ',inplace=True)
ex_omsk_KZ['code'].replace(['1601009900'],'ПРОЧИЕ ГОТОВЫЕ ПИЩЕВЫЕ ПРОДУКТЫ',inplace=True)
ex_omsk_KZ['code'].replace(['0402991000'],'МОЛОКО И СЛИВКИ, СГУЩЕННЫЕ',inplace=True)
ex_omsk_KZ['code'].replace(['8432311900'],'СЕЯЛКИ ПРОЧИЕ, ДЛЯ БЕСПАХАТНОЙ (ПОЧВОСБЕРЕГАЮЩЕЙ) СИСТЕМЫ',inplace=True)
ex_omsk_KZ['code'].replace(['1901909900'],'ПРОЧИЕ ГОТОВЫЕ ПИЩ. ПРОДУКТЫ ИЗ МУКИ',inplace=True)
ex_omsk_KZ['code'].replace(['9403700008'],'ПРОЧАЯ МЕБЕЛЬ ИЗ ПЛАСТМАССЫ',inplace=True)
ex_omsk_KZ['code'].replace(['9031803800'],'ПРОЧИЕ ПРИБОРЫ, УСТРОЙСТВА И МАШИНЫ ЭЛЕКТРОННЫЕ',inplace=True)
ex_omsk_KZ['code'].replace(['1905319900'],'ПРОЧЕЕ СЛАДКОЕ СУХОЕ ПЕЧЕНЬЕ',inplace=True)
ex_omsk_KZ['code'].replace(['8537109900'],'ПУЛЬТЫ, ПАНЕЛИ, КОНСОЛИ, СТОЛЫ, РАСПРЕДЕЛИТЕЛЬНЫЕ ЩИТЫ',inplace=True)
ex_omsk_KZ['code'].replace(['4819100000'],'ЯЩИКИ И КОРОБКИ ИЗ ГОФРИРОВАННОЙ БУМАГИ ИЛИ КАРТОНА',inplace=True)
ex_omsk_KZ['code'].replace(['1806901900'],'ПРОЧИЕ ШОКОЛАДНЫЕ КОНФЕТЫ',inplace=True)
ex_omsk_KZ['code'].replace(['8531103000'],'УСТРОЙСТВА СИГНАЛИЗАЦИОННЫЕ ОХРАННЫЕ, ДЛЯ ПОДАЧИ ПОЖАРНОГО СИГНАЛА',inplace=True)
ex_omsk_KZ['code'].replace(['2203000100'],'ПИВО СОЛОДОВОЕ В СОСУДАХ ЕМКОСТЬЮ 10 Л. ИЛИ МЕНЕЕ',inplace=True)
ex_omsk_KZ['code'].replace(['1001912000'],'ПШЕНИЦА МЯГКАЯ И МЕСЛИН',inplace=True)
ex_omsk_KZ['code'].replace(['1001190000'],'ПШЕНИЦА ТВЕРДАЯ ПРОЧАЯ',inplace=True)
ex_omsk_KZ['code'].replace(['2203000900'],'ПРОЧЕЕ ПИВО СОЛОДОВОЕ, В СОСУДАХ ЕМКОСТЬЮ 10 Л ИЛИ МЕНЕЕ',inplace=True)

In [ ]:
fig = px.scatter(ex_omsk_KZ, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Экспорт товаров в Казахстан 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
ex_omsk_UZ=data[(data['region'] == 52) & (data['direction'] == 'ЭК') & (data['country'] == 'UZ')]
ex_omsk_UZ = ex_omsk_UZ.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
ex_omsk_UZ = ex_omsk_UZ.sort_values('value', axis=0, ascending=False)
ex_omsk_UZ = ex_omsk_UZ.head(20)
ex_omsk_UZ.round({'value': 1, 'netto': 1})

,code,value,netto
53,2709009009,17102133.2,35861573.0
92,4011209000,12064992.2,6641350.0
22,1507109009,5607599.4,7251290.0
109,4707100000,5426175.8,32464541.5
220,8479899708,5041691.6,196127.2
100,4407119300,2056242.2,13206550.0
55,2803000000,1797267.5,1838384.0
29,1806901900,1748651.0,1550192.3
16,1003900000,1653893.0,8432800.0
199,8458112009,1261920.0,73380.0


In [ ]:
ex_omsk_UZ['code'].replace(['2709009009'],'НЕФТЬ СЫРАЯ (REBCO)',inplace=True)
ex_omsk_UZ['code'].replace(['4011209000'],'ШИНЫ И ПОКРЫШКИ ПНЕВМАТИЧЕСКИЕ РЕЗИНОВЫЕ НОВЫЕ (АВТОБУСЫ)',inplace=True)
ex_omsk_UZ['code'].replace(['1507109009'],'ПРОЧЕЕ МАСЛО СОЕВОЕ СЫРОЕ',inplace=True)
ex_omsk_UZ['code'].replace(['4707100000'],'НЕБЕЛЕНАЯ КРАФТ-БУМАГА ИЛИ КАРТОН',inplace=True)
ex_omsk_UZ['code'].replace(['8479899708'],'ИЗДЕЛИЯ ДЛЯ ГРАЖД. ВОЗД. СУДОВ',inplace=True)
ex_omsk_UZ['code'].replace(['4407119300'],'СОСНА ОБЫКНОВЕННАЯ',inplace=True)
ex_omsk_UZ['code'].replace(['2803000000'],'УГЛЕРОД (САЖИ И ПРОЧИЕ ФОРМЫ УГЛЕРОДА)',inplace=True)
ex_omsk_UZ['code'].replace(['1806901900'],'ПРОЧИЕ ШОКОЛАДНЫЕ КОНФЕТЫ',inplace=True)
ex_omsk_UZ['code'].replace(['1003900000'],'ЯЧМЕНЬ ПРОЧИЙ, ПИВОВАРЕННЫЙ',inplace=True)
ex_omsk_UZ['code'].replace(['8458112009'],'ГОРИЗОНТАЛЬНЫЕ СТАНКИ ТОКАРНЫЕ МНОГОЦЕЛЕВЫЕ',inplace=True)
ex_omsk_UZ['code'].replace(['4011100003'],'ШИНЫ И ПОКРЫШКИ ПНЕВМАТИЧЕСКИЕ РЕЗИНОВЫЕ НОВЫЕ (ЛЕГКОВЫЕ)',inplace=True)
ex_omsk_UZ['code'].replace(['1512119109'],'ПРОЧЕЕ МАСЛО ПОДСОЛНЕЧНОЕ, СЫРОЕ',inplace=True)
ex_omsk_UZ['code'].replace(['4407129800'],'ПРОЧИЕ ЛЕСОМАТЕРИАЛЫ ИЗ ПИХТЫ',inplace=True)
ex_omsk_UZ['code'].replace(['2930909509'],'ТИОМОЧЕВИНА, 99%,',inplace=True)
ex_omsk_UZ['code'].replace(['4002199000'],'ПРОЧИЙ КАУЧУК',inplace=True)
ex_omsk_UZ['code'].replace(['3916909000'],'ПЛАСТИКОВЫЙ ПРУТОК',inplace=True)
ex_omsk_UZ['code'].replace(['4707909000'],'ПРОЧИЕ ОТСОРТИРОВАННЫЕ БУМАЖНЫЕ ИЛИ КАРТОННЫЕ ОТХОДЫ И МАКУЛАТУРА.',inplace=True)
ex_omsk_UZ['code'].replace(['4011201000'],'ШИНЫ ДЛЯ АВТОБУСОВ ИЛИ МОТОРНЫХ ТРАНСПОРТНЫХ СРЕДСТВ',inplace=True)
ex_omsk_UZ['code'].replace(['9031200000'],'СТЕНДЫ ИСПЫТАТЕЛЬНЫЕ',inplace=True)
ex_omsk_UZ['code'].replace(['8458918009'],'ПРОЧИЕ ТОКАРНЫЕ СТАНКИ С ЧИСЛОВЫМ ПРОГРАММНЫМ УПРАВЛЕНИЕМ',inplace=True)

In [ ]:
fig = px.scatter(ex_omsk_UZ, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Экспорт товаров в Узбекистан 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
ex_omsk_KG=data[(data['region'] == 52) & (data['direction'] == 'ЭК') & (data['country'] == 'KG')]
ex_omsk_KG = ex_omsk_KG.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
ex_omsk_KG = ex_omsk_KG.sort_values('value', axis=0, ascending=False)
ex_omsk_KG = ex_omsk_KG.head(20)
ex_omsk_KG.round({'value': 1, 'netto': 1})

,code,value,netto
40,1001990000,19232127.2,84951767.0
111,1905319900,4920082.9,5234341.9
341,4011209000,4338598.4,2323641.6
143,2203000100,4312659.0,9832833.2
113,1905329900,3151871.6,2665924.0
338,4011100003,2916388.7,1295241.9
90,1806901900,2516720.1,1604998.7
310,3923301010,1864139.4,1400962.8
277,3916909000,1819587.1,512099.5
157,2710194210,1776869.0,4998115.0


In [ ]:
ex_omsk_KG['code'].replace(['1001990000'],'ПРОЧИЕ ПШЕНИЦА И МЕСЛИН',inplace=True)
ex_omsk_KG['code'].replace(['1905319900'],'ПРОЧЕЕ СЛАДКОЕ СУХОЕ ПЕЧЕНЬЕ',inplace=True)
ex_omsk_KG['code'].replace(['4011209000'],'ШИНЫ И ПОКРЫШКИ ПНЕВМАТИЧЕСКИЕ РЕЗИНОВЫЕ НОВЫЕ (АВТОБУСЫ)',inplace=True)
ex_omsk_KG['code'].replace(['2203000100'],'ПИВО СОЛОДОВОЕ В СОСУДАХ ЕМКОСТЬЮ 10 Л. ИЛИ МЕНЕЕ',inplace=True)
ex_omsk_KG['code'].replace(['1905329900'],'ВАФЛИ И ВАФЕЛЬНЫЕ ОБЛАТКИ',inplace=True)
ex_omsk_KG['code'].replace(['4011100003'],'ШИНЫ И ПОКРЫШКИ ПНЕВМАТИЧЕСКИЕ РЕЗИНОВЫЕ НОВЫЕ (ЛЕГКОВЫЕ)',inplace=True)
ex_omsk_KG['code'].replace(['1806901900'],'ПРОЧИЕ ШОКОЛАДНЫЕ КОНФЕТЫ',inplace=True)
ex_omsk_KG['code'].replace(['3923301010'],'БУТЫЛИ И АНАЛОГИЧНЫЕ ИЗДЕЛИЯ, ЕМКОСТЬЮ НЕ БОЛЕЕ 2 Л, ИЗ ПЛАСТМАСС',inplace=True)
ex_omsk_KG['code'].replace(['3916909000'],'ПЛАСТИКОВЫЙ ПРУТОК',inplace=True)
ex_omsk_KG['code'].replace(['2710194210'],'ГАЗОЙЛИ ДЛЯ ПРОЧИХ ЦЕЛЕЙ',inplace=True)
ex_omsk_KG['code'].replace(['0713400000'],'ЧЕЧЕВИЦА СУШЕНАЯ',inplace=True)
ex_omsk_KG['code'].replace(['4411139000'],'ПЛИТЫ ДРЕВЕСНО-ВОЛОКНИСТЫЕ СРЕДНЕЙ ПЛОТНОСТИ (MDF), 5 ММ',inplace=True)
ex_omsk_KG['code'].replace(['2711129700'],'ПРОЧИЙ СЖИЖЕННЫЙ ПРОПАН',inplace=True)
ex_omsk_KG['code'].replace(['7010901009'],'ПРОЧИЕ БАНКИ ДЛЯ КОНСЕРВИРОВАНИЯ СТЕКЛЯННЫЕ.',inplace=True)
ex_omsk_KG['code'].replace(['1901909900'],'ПРОЧИЕ ГОТОВЫЕ ПИЩ. ПРОДУКТЫ ИЗ МУКИ',inplace=True)
ex_omsk_KG['code'].replace(['2901100001'],'УГЛЕВОДОРОДЫ АЦИКЛИЧЕСКИЕ НАСЫЩЕННЫЕ',inplace=True)
ex_omsk_KG['code'].replace(['1003900000'],'ЯЧМЕНЬ ПРОЧИЙ, ПИВОВАРЕННЫЙ',inplace=True)
ex_omsk_KG['code'].replace(['2203000900'],'ПРОЧЕЕ ПИВО СОЛОДОВОЕ, В СОСУДАХ ЕМКОСТЬЮ 10 Л ИЛИ МЕНЕЕ',inplace=True)
ex_omsk_KG['code'].replace(['2909199000'],'ЭФИРЫ ПРОСТЫЕ АЦИКЛИЧЕСКИЕ',inplace=True)
ex_omsk_KG['code'].replace(['4411149000'],'ПЛИТЫ ДРЕВЕСНО-ВОЛОКНИСТЫЕ СРЕДНЕЙ ПЛОТНОСТИ (MDF), 9 ММ',inplace=True)

In [ ]:
fig = px.scatter(ex_omsk_KG, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Экспорт товаров в Киргизию 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
ex_omsk_IR=data[(data['region'] == 52) & (data['direction'] == 'ЭК') & (data['country'] == 'IR')]
ex_omsk_IR = ex_omsk_IR.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
ex_omsk_IR = ex_omsk_IR.sort_values('value', axis=0, ascending=False)
ex_omsk_IR = ex_omsk_IR.head(20)
ex_omsk_IR.round({'value': 1, 'netto': 1})

,code,value,netto
7,XXXXXXXXXX,34654486.3,142167862.0
1,1003900000,3789080.2,18876976.0
0,0713400000,542215.6,1484300.0
3,4407119300,87128.0,253996.0
4,4407969001,15567.0,42304.0
2,3911100000,11809.0,18000.0
5,4417000000,6663.0,11611.0
6,8609009009,2441.0,6700.0


In [ ]:
ex_omsk_IR['code'].replace(['XXXXXXXXXX'],'XXXXXXXXXX',inplace=True)
ex_omsk_IR['code'].replace(['1003900000'],'ЯЧМЕНЬ ПРОЧИЙ, ПИВОВАРЕННЫЙ',inplace=True)
ex_omsk_IR['code'].replace(['0713400000'],'ЧЕЧЕВИЦА СУШЕНАЯ',inplace=True)
ex_omsk_IR['code'].replace(['4407119300'],'СОСНА ОБЫКНОВЕННАЯ',inplace=True)
ex_omsk_IR['code'].replace(['4407969001'],'ЛЕСОМАТЕРИАЛЫ ИЗ БЕРЕЗЫ',inplace=True)
ex_omsk_IR['code'].replace(['3911100000'],'СМОЛЫ НЕФТЯНЫЕ',inplace=True)
ex_omsk_IR['code'].replace(['4417000000'],'ИНСТРУМЕНТЫ, КОРПУСА И РУЧКИ ДЛЯ ИНСТРУМЕНТОВ',inplace=True)
ex_omsk_IR['code'].replace(['8609009009'],'ПРОЧИЕ КОНТЕЙНЕРЫ И ЕМКОСТИ',inplace=True)

In [ ]:
fig = px.scatter(ex_omsk_IR, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Экспорт товаров в Иран 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
ex_omsk_IN=data[(data['region'] == 52) & (data['direction'] == 'ЭК') & (data['country'] == 'IN')]
ex_omsk_IN = ex_omsk_IN.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
ex_omsk_IN = ex_omsk_IN.sort_values('value', axis=0, ascending=False)
ex_omsk_IN = ex_omsk_IN.head(20)
ex_omsk_IN.round({'value': 1, 'netto': 1})

,code,value,netto
8,4002199000,13789729.8,11422620.0
2,2803000000,12471974.7,21382297.0
38,8413910008,1565725.7,56.4
5,3902100000,1552947.0,1810875.0
48,8481807399,1131892.8,2.2
3,2902900000,1128109.6,1063200.0
17,4421999900,425257.2,206678.1
46,8481401000,420205.0,0.6
39,8421230000,417111.2,5.7
9,4016930005,404929.6,1.7


In [ ]:
ex_omsk_IN['code'].replace(['4002199000'],'ПРОЧИЙ КАУЧУК',inplace=True)
ex_omsk_IN['code'].replace(['2803000000'],'УГЛЕРОД',inplace=True)
ex_omsk_IN['code'].replace(['8413910008'],'НАСОСЫ ЖИДКОСТНЫЕ ПРОЧИЕ',inplace=True)
ex_omsk_IN['code'].replace(['3902100000'],'ПОЛИПРОПИЛЕН ',inplace=True)
ex_omsk_IN['code'].replace(['8481807399'],'АРМАТУРА ЗАПОРНАЯ ТРУБОПРОВОДНАЯ',inplace=True)
ex_omsk_IN['code'].replace(['2902900000'],'УГЛЕВОДОРОДЫ ЦИКЛИЧЕСКИЕ: КСИЛОЛЫ',inplace=True)
ex_omsk_IN['code'].replace(['4421999900'],'ПРОЧИЕ ДЕРЕВЯННЫЕ ИЗДЕЛИЯ',inplace=True)
ex_omsk_IN['code'].replace(['8481401000'],'КЛАПАНЫ ПРЕДОХРАНИТЕЛЬНЫЕ ИЛИ РАЗГРУЗОЧНЫЕ',inplace=True)
ex_omsk_IN['code'].replace(['8421230000'],'ОБОРУДОВАНИЕ ДЛЯ ФИЛЬТРОВАНИЯ МАСЛА ИЛИ ТОПЛИВА',inplace=True)
ex_omsk_IN['code'].replace(['4016930005'],'ПРОЧИЕ ПРОКЛАДКИ, ШАЙБЫ, УПЛОТНИТЕЛИ',inplace=True)
ex_omsk_IN['code'].replace(['8481101908'],'КЛАПАНЫ РЕДУКЦИОННЫЕ ДЛЯ РЕГУЛИРОВКИ ДАВЛЕНИЯ',inplace=True)
ex_omsk_IN['code'].replace(['8481805910'],'ПРОЧИЕ РЕГУЛЯТОРЫ ДАВЛЕНИЯ',inplace=True)
ex_omsk_IN['code'].replace(['7609000000'],'ФИТИНГИ ДЛЯ ТРУБ И ТРУБОК АЛЮМИНИЕВЫЕ',inplace=True)
ex_omsk_IN['code'].replace(['8481109909'],'АРМАТУРА ПРОМЫШЛЕННАЯ ТРУБОПРОВОДНАЯ',inplace=True)
ex_omsk_IN['code'].replace(['7616991009'],'ФОРМООБРАЗУЮЩИЙ ЛИТОЙ ЭЛЕМЕНТ ИЗ АЛЮМИНИЯ',inplace=True)
ex_omsk_IN['code'].replace(['7326909807'],'ПРОЧИЕ ИЗДЕЛИЯ ИЗ ЧЕРНОГО МЕТАЛЛА',inplace=True)
ex_omsk_IN['code'].replace(['2907110000'],'ФЕНОЛЫ',inplace=True)
ex_omsk_IN['code'].replace(['8204110000'],'РУЧНЫЕ ИНСТРУМЕНТЫ',inplace=True)
ex_omsk_IN['code'].replace(['4407119300'],'СОСНА ОБЫКНОВЕННАЯ',inplace=True)
ex_omsk_IN['code'].replace(['7616100000'],'ГВОЗДКИ, КНОПКИ, СКОБЫ',inplace=True)

In [ ]:
fig = px.scatter(ex_omsk_IN, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Экспорт товаров в Индию 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
ex_omsk_PK=data[(data['region'] == 52) & (data['direction'] == 'ЭК') & (data['country'] == 'PK')]
ex_omsk_PK = ex_omsk_PK.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
ex_omsk_PK = ex_omsk_PK.sort_values('value', axis=0, ascending=False)
ex_omsk_PK = ex_omsk_PK.head(20)
ex_omsk_PK.round({'value': 1, 'netto': 1})

,code,value,netto
4,4002199000,6561749.3,5396580.0
3,2803000000,2366577.0,4001754.0
1,0713400000,316241.2,701200.0
0,0713109009,312218.0,1152520.0
2,1004900000,80559.6,398810.0


In [ ]:
ex_omsk_PK['code'].replace(['4002199000'],'ПРОЧИЙ КАУЧУК',inplace=True)
ex_omsk_PK['code'].replace(['2803000000'],'УГЛЕРОД ',inplace=True)
ex_omsk_PK['code'].replace(['0713400000'],'ЧЕЧЕВИЦА СУШЕНАЯ',inplace=True)
ex_omsk_PK['code'].replace(['0713109009'],'ГОРОХ',inplace=True)
ex_omsk_PK['code'].replace(['1004900000'],'ОВЕС ПРОЧИЙ',inplace=True)

In [ ]:
fig = px.scatter(ex_omsk_PK, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Экспорт товаров в Пакистан 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
ex_omsk_CN=data[(data['region'] == 52) & (data['direction'] == 'ЭК') & (data['country'] == 'CN')]
ex_omsk_CN = ex_omsk_CN.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
ex_omsk_CN = ex_omsk_CN.sort_values('value', axis=0, ascending=False)
ex_omsk_CN = ex_omsk_CN.head(20)
ex_omsk_CN.round({'value': 1, 'netto': 1})

,code,value,netto
28,1204009000,128797613.9,257912374.8
44,1514119009,86969259.0,91979823.0
136,4002199000,81312556.9,67269660.0
29,1205109000,20342609.8,49840644.4
114,2709009009,15909573.9,32450483.0
131,3902100000,8438229.1,9330620.0
220,7601208002,6300452.8,3314568.0
42,1514111000,5602584.2,7081970.0
57,1806901900,5192451.3,3103687.3
22,1003900000,4626887.8,19494620.0


In [ ]:
ex_omsk_CN['code'].replace(['1204009000'],'ПРОЧИЕ СЕМЕНА ЛЬНА',inplace=True)
ex_omsk_CN['code'].replace(['1514119009'],'ПРОЧЕЕ МАСЛО РАПСОВОЕ ДЛЯ ТЕХНИЧЕСКОГО ПРИМЕНЕНИЯ',inplace=True)
ex_omsk_CN['code'].replace(['4002199000'],'ПРОЧИЙ КАУЧУК',inplace=True)
ex_omsk_CN['code'].replace(['1205109000'],'ПРОЧИЕ СЕМЕНА РАПСА',inplace=True)
ex_omsk_CN['code'].replace(['2709009009'],'НЕФТЬ СЫРАЯ И НЕФТЕПРОДУКТЫ СЫРЫЕ ',inplace=True)
ex_omsk_CN['code'].replace(['3902100000'],'ПОЛИПРОПИЛЕН',inplace=True)
ex_omsk_CN['code'].replace(['7601208002'],'СПЛАВЫ АЛЮМИНИЕВЫЕ, ПРОЧИЕ',inplace=True)
ex_omsk_CN['code'].replace(['1514111000'],'МАСЛО РАПСОВОЕ СЫРОЕ',inplace=True)
ex_omsk_CN['code'].replace(['1806901900'],'ПРОЧИЕ ШОКОЛАДНЫЕ КОНФЕТЫ',inplace=True)
ex_omsk_CN['code'].replace(['1003900000'],'ЯЧМЕНЬ ПРОЧИЙ',inplace=True)
ex_omsk_CN['code'].replace(['2902900000'],'УГЛЕВОДОРОДЫ ЦИКЛИЧЕСКИЕ',inplace=True)
ex_omsk_CN['code'].replace(['1206009900'],'ПРОЧИЕ СЕМЕНА ПОДСОЛНЕЧНИКА',inplace=True)
ex_omsk_CN['code'].replace(['0511919090'],'ПРОЧИЕ ОТХОДЫ РЫБНЫЕ',inplace=True)
ex_omsk_CN['code'].replace(['1512119109'],'ПРОЧЕЕ МАСЛО ПОДСОЛНЕЧНОЕ, СЫРОЕ',inplace=True)
ex_omsk_CN['code'].replace(['2208601100'],'ВОДКА, С КОНЦЕНТРАЦИЕЙ СПИРТА 45',inplace=True)
ex_omsk_CN['code'].replace(['1205900009'],'ПРОЧИЕ СЕМЕНА РАПСА',inplace=True)
ex_omsk_CN['code'].replace(['1704909900'],'ПРОЧИЕ КОНДИТЕРСКИЕ ИЗДЕЛИЯ ИЗ САХАРА',inplace=True)
ex_omsk_CN['code'].replace(['1207509000'],'ПРОЧИЕ СЕМЕНА ГОРЧИЦЫ',inplace=True)
ex_omsk_CN['code'].replace(['2711129400'],'СЖИЖЕННЫЙ ПРОПАН ЧИСТОТОЙ БОЛЕЕ 90%',inplace=True)
ex_omsk_CN['code'].replace(['1507109009'],'ПРОЧЕЕ МАСЛО СОЕВОЕ СЫРОЕ',inplace=True)

In [ ]:
fig = px.scatter(ex_omsk_CN, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Экспорт товаров в Китай 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
ex_omsk_MN=data[(data['region'] == 52) & (data['direction'] == 'ЭК') & (data['country'] == 'MN')]
ex_omsk_MN = ex_omsk_MN.groupby(['code'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
ex_omsk_MN = ex_omsk_MN.sort_values('value', axis=0, ascending=False)
ex_omsk_MN = ex_omsk_MN.head(20)
ex_omsk_MN.round({'value': 1, 'netto': 1})

,code,value,netto
11,1205109000,9912373.9,25959873.8
6,1001990000,9157231.8,43582075.0
28,1905329900,7522227.3,5993978.7
38,2711129700,5145840.7,9863962.0
10,1204009000,5099401.2,13465524.0
43,2901100001,4968643.0,11630046.0
19,1806901900,4003648.5,1853419.3
37,2711129400,3981257.2,8891400.0
42,2713200000,3192035.4,8252291.0
44,2909199000,1857495.5,2865023.0


In [ ]:
ex_omsk_MN['code'].replace(['1205109000'],'ПРОЧИЕ СЕМЕНА РАПСА',inplace=True)
ex_omsk_MN['code'].replace(['1001990000'],'ПШЕНИЦА И МЕСЛИН',inplace=True)
ex_omsk_MN['code'].replace(['1905329900'],'ВАФЕЛЬНЫЕ ТРУБОЧКИ',inplace=True)
ex_omsk_MN['code'].replace(['2711129700'],'ГАЗ СЖИЖЕННЫЙ, ШИРОКАЯ ФРАКЦИЯ',inplace=True)
ex_omsk_MN['code'].replace(['1204009000'],'ПРОЧИЕ СЕМЕНА ЛЬНА',inplace=True)
ex_omsk_MN['code'].replace(['2901100001'],'ИЗОБУТАН',inplace=True)
ex_omsk_MN['code'].replace(['1806901900'],'ПРОЧИЕ ШОКОЛАДНЫЕ КОНФЕТЫ',inplace=True)
ex_omsk_MN['code'].replace(['2711129400'],'СЖИЖЕННЫЙ ПРОПАН ЧИСТОТОЙ БОЛЕЕ 90%',inplace=True)
ex_omsk_MN['code'].replace(['2713200000'],'БИТУМ НЕФТЯНОЙ',inplace=True)
ex_omsk_MN['code'].replace(['2909199000'],'ЭФИРЫ ПРОСТЫЕ АЦИКЛИЧЕСКИЕ И ИХ ГАЛОГЕНИРОВАННЫЕ',inplace=True)
ex_omsk_MN['code'].replace(['8432291000'],'РЫХЛИТЕЛИ И КУЛЬТИВАТОРЫ.',inplace=True)
ex_omsk_MN['code'].replace(['1901909900'],'ПРОЧИЕ ГОТОВЫЕ ПИЩ. ПРОДУКТЫ ИЗ МУКИ',inplace=True)
ex_omsk_MN['code'].replace(['1207509000'],'ПРОЧИЕ СЕМЕНА ГОРЧИЦЫ',inplace=True)
ex_omsk_MN['code'].replace(['8432391900'],'ПРОЧИЕ СЕЯЛКИ, ПРОЧИЕ',inplace=True)
ex_omsk_MN['code'].replace(['1905319900'],'ПРОЧЕЕ СЛАДКОЕ СУХОЕ ПЕЧЕНЬЕ.',inplace=True)
ex_omsk_MN['code'].replace(['1205900009'],'ПРОЧИЕ СЕМЕНА РАПСА, ИЛИ КОЛЬЗЫ',inplace=True)
ex_omsk_MN['code'].replace(['1902301000'],'ПРОЧИЕ МАКАРОННЫЕ ИЗДЕЛИЯ, СУШЕНЫЕ',inplace=True)
ex_omsk_KZ['code'].replace(['3921110000'],'ПЛИТЫ, ЛИСТЫ, ПЛЕНКИ, ФОЛЬГА И ПОЛОСА, ПОРИСТЫЕ ИЗ ПОЛИМЕРОВ СТИРОЛА',inplace=True)
ex_omsk_MN['code'].replace(['0406303100'],'ПРОЧИЕ ПЛАВЛЕННЫЕ СЫРЫ',inplace=True)
ex_omsk_MN['code'].replace(['1704906500'],'КОНДИТЕРСКИЕ ИЗДЕЛИЯ В ВИДЕ РЕЗИНКИ И ЖЕЛЕ',inplace=True)

In [ ]:
fig = px.scatter(ex_omsk_MN, x='netto', y='value', color='code')
fig.update_layout(height=400, width=1200,
                  title='Экспорт товаров в Монголию 2016-2021', title_x = 0.3,
                  xaxis_title='Вес товаров (кг)',
                  yaxis_title='Стоимость товаров ($)',
                  margin=dict(l=0, r=0, t=30, b=0),
                  legend_title_text='Товар',
                  legend_tracegroupgap=0)
fig.show()

#***РАЗДЕЛ 3***

In [ ]:
im_omsk_0_1=data[(data['region'] == 52) & (data['direction'] == 'ИМ')] #Омск
im_omsk_0_2=data[(data['region'] == 50) & (data['direction'] == 'ИМ')] #Новосибирск
im_omsk_0_3=data[(data['region'] == 1) & (data['direction'] == 'ИМ')] #Алтай
im_omsk_0_4=data[(data['region'] == 37) & (data['direction'] == 'ИМ')] #Курган
im_omsk_0_5=data[(data['region'] == 75) & (data['direction'] == 'ИМ')] #Челябинск
im_omsk_0_6=data[(data['region'] == 53) & (data['direction'] == 'ИМ')] #Оренбург
im_omsk_0=pd.concat([im_omsk_0_1, im_omsk_0_2, im_omsk_0_3, im_omsk_0_4, im_omsk_0_5, im_omsk_0_6], sort=False)
im_omsk_0['month'] = pd.to_datetime(im_omsk_0.month, dayfirst=True)
im_omsk_0['region'].replace([52],'OMS',inplace=True)
im_omsk_0['region'].replace([50],'NVS',inplace=True)
im_omsk_0['region'].replace([1],'ALT',inplace=True)
im_omsk_0['region'].replace([37],'KGN',inplace=True)
im_omsk_0['region'].replace([75],'CHE',inplace=True)
im_omsk_0['region'].replace([53],'ORE',inplace=True)
im_omsk_0

,direction,month,country,code,measure,value,netto,quantity,region,district
1651,ИМ,2016-01-01,CN,8413910008,1,1015.00,150.0,0.0,OMS,6
1926,ИМ,2016-01-01,BY,6108220000,ШТ,496.00,6.0,198.0,OMS,6
2447,ИМ,2016-01-01,JP,8407330000,ШТ,516.00,344.0,2.0,OMS,6
2972,ИМ,2016-01-01,JP,5705003000,М2,980.00,140.0,34.0,OMS,6
3154,ИМ,2016-01-01,JP,8512909009,1,2000.00,80.0,0.0,OMS,6
...,...,...,...,...,...,...,...,...,...,...
21855711,ИМ,2021-12-01,BY,8412218008,ШТ,4770.30,340.0,18.0,ORE,4
21855728,ИМ,2021-12-01,KZ,1905329900,1,8189.13,6044.8,0.0,ORE,4
21855733,ИМ,2021-12-01,KZ,1806310000,1,851.70,1940.0,0.0,ORE,4
21855745,ИМ,2021-12-01,KZ,1806906000,1,1946.37,1008.0,0.0,ORE,4


In [ ]:
#ИМ. Молоко сливки, код - 0401209109, 0401201109, 0402991000
im_omsk_1_1=im_omsk_0[(im_omsk_0['code'] == '0401209109')]
im_omsk_1_2=im_omsk_0[(im_omsk_0['code'] == '0401201109')]
im_omsk_1_3=im_omsk_0[(im_omsk_0['code'] == '0402991000')]
im_omsk_1=pd.concat([im_omsk_1_1, im_omsk_1_2, im_omsk_1_3], sort=False)
im_omsk_1=im_omsk_1.groupby(['region', 'month'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'}) 
im_omsk_1=im_omsk_1.sort_values('month', axis=0, ascending=True)
im_omsk_1.round({'value': 1, 'netto': 1})
fig = px.line(im_omsk_1, x='month', y='netto', markers=True, color='region')
fig.update_layout(height=600, width=1500,
                  title='Импорт молока и сливок в 2016-2021 (Казахстан)', title_x = 0.3,
                  xaxis_title='Временной интервал',
                  yaxis_title='Вес товаров (кг)',
                  legend_title_text='Регион РФ',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
#ИМ. Мясо птицы, мясо крс, код - 0207119009, 0201300004
im_omsk_2_1=im_omsk_0[(im_omsk_0['code'] == '0207119009')]
im_omsk_2_2=im_omsk_0[(im_omsk_0['code'] == '0201300004')]
im_omsk_2=pd.concat([im_omsk_2_1, im_omsk_2_2], sort=False)
im_omsk_2=im_omsk_2.groupby(['region', 'month'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_2=im_omsk_2.sort_values('month', axis=0, ascending=True)
im_omsk_2.round({'value': 1, 'netto': 1})
fig = px.line(im_omsk_2, x='month', y='netto', markers=True, color='region')
fig.update_layout(height=600, width=1500,
                  title='Импорт мяса птицы, КРС в 2016-2021 (Казахстан)', title_x = 0.3,
                  xaxis_title='Временной интервал',
                  yaxis_title='Вес товаров (кг)',
                  legend_title_text='Регион РФ',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
#ИМ. Форель, код - 0303141000,0302111000, 0303142000, 0303149000, 0302112000, 0405101100
im_omsk_3_1=im_omsk_0[(im_omsk_0['code'] == '0303141000')]
im_omsk_3_2=im_omsk_0[(im_omsk_0['code'] == '0302111000')]
im_omsk_3_3=im_omsk_0[(im_omsk_0['code'] == '0303142000')]
im_omsk_3_4=im_omsk_0[(im_omsk_0['code'] == '0303149000')]
im_omsk_3_5=im_omsk_0[(im_omsk_0['code'] == '0302112000')]
im_omsk_3_6=im_omsk_0[(im_omsk_0['code'] == '0405101100')]
im_omsk_3=pd.concat([im_omsk_3_1, im_omsk_3_2, im_omsk_3_3, im_omsk_3_4, im_omsk_3_5, im_omsk_3_6], sort=False)
im_omsk_3=im_omsk_3.groupby(['region', 'month'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_3=im_omsk_3.sort_values('month', axis=0, ascending=True)
im_omsk_3.round({'value': 1, 'netto': 1})
fig = px.line(im_omsk_3, x='month', y='netto', markers=True, color='region')
fig.update_layout(height=600, width=1500,
                  title='Импорт форели в 2016-2021 (Киргизия)', title_x = 0.3,
                  xaxis_title='Временной интервал',
                  yaxis_title='Вес товаров (кг)',
                  legend_title_text='Регион РФ',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
#ИМ. Масло (сливочное, подсолнечное), код - 0405101900, 0405101100, 1512119109
im_omsk_4_1=im_omsk_0[(im_omsk_0['code'] == '0405101900')]
im_omsk_4_2=im_omsk_0[(im_omsk_0['code'] == '0405101100')]
im_omsk_4_3=im_omsk_0[(im_omsk_0['code'] == '1512119109')]
im_omsk_4=pd.concat([im_omsk_4_1, im_omsk_4_2, im_omsk_4_3], sort=False)
im_omsk_4=im_omsk_4.groupby(['region', 'month'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_4=im_omsk_4.sort_values('month', axis=0, ascending=True)
im_omsk_4.round({'value': 1, 'netto': 1})
fig = px.line(im_omsk_4, x='month', y='netto', markers=True, color='region')
fig.update_layout(height=600, width=1500,
                  title='Импорт масла (сливочное, подсолнечное) в 2016-2021 (Киргизия)', title_x = 0.3,
                  xaxis_title='Временной интервал',
                  yaxis_title='Вес товаров (кг)',
                  legend_title_text='Регион РФ',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
#ИМ. Морковь, код - 0706100001
im_omsk_5=im_omsk_0[(im_omsk_0['code'] == '0706100001')]
im_omsk_5=im_omsk_5.groupby(['region', 'month'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_5=im_omsk_5.sort_values('month', axis=0, ascending=True)
im_omsk_5.round({'value': 1, 'netto': 1})
fig = px.line(im_omsk_5, x='month', y='netto', markers=True, color='region')
fig.update_layout(height=600, width=1500,
                  title='Импорт моркови в 2016-2021 (Киргизия)', title_x = 0.3,
                  xaxis_title='Временной интервал',
                  yaxis_title='Вес товаров (кг)',
                  legend_title_text='Регион РФ',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
#ИМ. Прочие овощи, код - 0704909000, 0813409500
im_omsk_6_1=im_omsk_0[(im_omsk_0['code'] == '0704909000')]
im_omsk_6_2=im_omsk_0[(im_omsk_0['code'] == '0813409500')]
im_omsk_6=pd.concat([im_omsk_6_1, im_omsk_4_2], sort=False)
im_omsk_6=im_omsk_6.groupby(['region', 'month'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_6=im_omsk_6.sort_values('month', axis=0, ascending=True)
im_omsk_6.round({'value': 1, 'netto': 1})
fig = px.line(im_omsk_6, x='month', y='netto', markers=True, color='region')
fig.update_layout(height=600, width=1500,
                  title='Импорт прочих овощей в 2016-2021 (Киргизия)', title_x = 0.3,
                  xaxis_title='Временной интервал',
                  yaxis_title='Вес товаров (кг)',
                  legend_title_text='Регион РФ',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
#ИМ. Томаты, код - 0702000001
im_omsk_7=im_omsk_0[(im_omsk_0['code'] == '0702000001')]
im_omsk_7=im_omsk_7.groupby(['region', 'month'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_7=im_omsk_7.sort_values('month', axis=0, ascending=True)
im_omsk_7.round({'value': 1, 'netto': 1})
fig = px.line(im_omsk_7, x='month', y='netto', markers=True, color='region')
fig.update_layout(height=600, width=1500,
                  title='Импорт томатов в 2016-2021 (Иран)', title_x = 0.3,
                  xaxis_title='Временной интервал',
                  yaxis_title='Вес товаров (кг)',
                  legend_title_text='Регион РФ',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
#ИМ. Горчица, код - 1207509000
im_omsk_8=im_omsk_0[(im_omsk_0['code'] == '1207509000')]
im_omsk_8=im_omsk_8.groupby(['region', 'month'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
im_omsk_8=im_omsk_8.sort_values('month', axis=0, ascending=True)
im_omsk_8.round({'value': 1, 'netto': 1})
fig = px.line(im_omsk_8, x='month', y='netto', markers=True, color='region')
fig.update_layout(height=600, width=1500,
                  title='Импорт горчицы в 2016-2021 (Монголия)', title_x = 0.3,
                  xaxis_title='Временной интервал',
                  yaxis_title='Вес товаров (кг)',
                  legend_title_text='Регион РФ',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
ex_omsk_0_1=data[(data['region'] == 52) & (data['direction'] == 'ЭК')] #Омск
ex_omsk_0_2=data[(data['region'] == 50) & (data['direction'] == 'ЭК')] #Новосибирск
ex_omsk_0_3=data[(data['region'] == 1) & (data['direction'] == 'ЭК')] #Алтай
ex_omsk_0_4=data[(data['region'] == 37) & (data['direction'] == 'ЭК')] #Курган
ex_omsk_0_5=data[(data['region'] == 75) & (data['direction'] == 'ЭК')] #Челябинск
ex_omsk_0_6=data[(data['region'] == 53) & (data['direction'] == 'ЭК')] #Оренбург
ex_omsk_0=pd.concat([ex_omsk_0_1, ex_omsk_0_2, ex_omsk_0_3, ex_omsk_0_4, ex_omsk_0_5, ex_omsk_0_6], sort=False)
ex_omsk_0['month'] = pd.to_datetime(ex_omsk_0.month, dayfirst=True)
ex_omsk_0['region'].replace([52],'OMS',inplace=True)
ex_omsk_0['region'].replace([50],'NVS',inplace=True)
ex_omsk_0['region'].replace([1],'ALT',inplace=True)
ex_omsk_0['region'].replace([37],'KGN',inplace=True)
ex_omsk_0['region'].replace([75],'CHE',inplace=True)
ex_omsk_0['region'].replace([53],'ORE',inplace=True)
ex_omsk_0

,direction,month,country,code,measure,value,netto,quantity,region,district
2448780,ЭК,2016-01-01,KZ,8539390000,ШТ,5290.00,74.000,1650.0,OMS,6
2448784,ЭК,2016-01-01,KZ,8467299000,ШТ,896.00,18.000,5.0,OMS,6
2448826,ЭК,2016-01-01,KZ,2007993909,1,266.00,283.000,0.0,OMS,6
2448827,ЭК,2016-01-01,KZ,6912005000,1,26.00,16.000,0.0,OMS,6
2448950,ЭК,2016-01-01,KZ,8440104000,ШТ,279.00,23.000,4.0,OMS,6
...,...,...,...,...,...,...,...,...,...,...
21855743,ЭК,2021-12-01,KZ,1602493000,1,29830.14,10487.428,0.0,ORE,4
21855744,ЭК,2021-12-01,KZ,0208909800,1,14827.31,5780.655,0.0,ORE,4
21855746,ЭК,2021-12-01,KZ,1604199780,1,526.50,285.900,0.0,ORE,4
21855747,ЭК,2021-12-01,KZ,1601009900,1,1037273.13,338285.375,0.0,ORE,4


In [ ]:
#ЭК. Чечевица, код - 0713400000
ex_omsk_9=ex_omsk_0[(ex_omsk_0['code'] == '0713400000')]
ex_omsk_9=ex_omsk_9.groupby(['region', 'month'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
ex_omsk_9=ex_omsk_9.sort_values('month', axis=0, ascending=True)
ex_omsk_9.round({'value': 1, 'netto': 1})
fig = px.line(ex_omsk_9, x='month', y='netto', markers=True, color='region')
fig.update_layout(height=600, width=1500,
                  title='Экспорт чечевицы в 2016-2021 (Киргизия, Иран, Пакистан)', title_x = 0.3,
                  xaxis_title='Временной интервал',
                  yaxis_title='Вес товаров (кг)',
                  legend_title_text='Регион РФ',
                  legend_tracegroupgap=0)
fig.show()

In [ ]:
#ЭК. Бутылки, банки - 7010904300, 7010901009
ex_omsk_10_1=ex_omsk_0[(ex_omsk_0['code'] == '7010904300')]
ex_omsk_10_2=ex_omsk_0[(ex_omsk_0['code'] == '7010901009')]
ex_omsk_10=pd.concat([ex_omsk_10_1, ex_omsk_10_2], sort=False)
ex_omsk_10=ex_omsk_10.groupby(['region', 'month'], as_index=False).agg({'value' : 'sum', 'netto' : 'sum'})
ex_omsk_10=ex_omsk_10.sort_values('month', axis=0, ascending=True)
ex_omsk_10.round({'value': 1, 'netto': 1})
fig = px.line(ex_omsk_10, x='month', y='netto', markers=True, color='region')
fig.update_layout(height=600, width=1500,
                  title='Экспорт стеклотары в 2016-2021 (Казахстан, Киргизия)', title_x = 0.3,
                  xaxis_title='Временной интервал',
                  yaxis_title='Вес товаров (кг)',
                  legend_title_text='Регион РФ',
                  legend_tracegroupgap=0)
fig.show()

#***РАЗДЕЛ 4*** (предсказания)

In [ ]:
ex_omsk_10

,region,month,value,netto
31,CHE,2016-01-01,13984.00,65964.00
167,OMS,2016-01-01,98702.00,507686.00
32,CHE,2016-02-01,4264.00,20033.00
168,OMS,2016-02-01,74121.00,350116.00
97,NVS,2016-02-01,153561.00,1077759.00
...,...,...,...,...
238,OMS,2021-12-01,988892.95,3168128.38
166,NVS,2021-12-01,54314.72,120813.30
89,CHE,2021-12-01,228.24,1526.00
96,KGN,2021-12-01,0.96,0.68
